In [1]:
import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, f1_score, precision_score, recall_score, confusion_matrix
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib


In [3]:
from google.colab import drive
drive.mount('/content/drive')

csv_path = "/content/drive/MyDrive/financial-fraud-dataset-train.csv"

# AE artefact'ları (daha önce kaydetmiştin)
ae_model_path = "/content/drive/MyDrive/improved_autoencoder.h5"      # adını sen nasıl kaydettiysen ona göre düzelt
scaler_path   = "/content/drive/MyDrive/ae_scaler.pkl"       # adını sen nasıl kaydettiysen ona göre düzelt


Mounted at /content/drive


In [4]:
AE_FEATURES = [
    "amount_ngn",
    "spending_deviation_score",
    "velocity_score",
    "user_avg_txn_amt",
    "user_std_txn_amt",
    "txn_hour",
    "is_night_txn",
    "user_txn_frequency_24h",
    "txn_count_last_1h",
    "avg_gap_between_txns",
    "device_seen_count",
    "is_device_shared",
    "new_device_transaction",
    "geospatial_velocity_anomaly"
]


In [5]:
LGBM_BASE_FEATURES = [
    "amount_ngn",
    "user_avg_txn_amt",
    "user_std_txn_amt",
    "user_txn_frequency_24h",
    "txn_count_last_1h",
    "txn_count_last_24h",
    "total_amount_last_1h",
    "avg_gap_between_txns",
    "txn_hour",
    "is_weekend",
    "is_night_txn",
    "device_seen_count",
    "is_device_shared",
    "ip_seen_count",
    "is_ip_shared",
    "new_device_transaction",
    "geospatial_velocity_anomaly",
    "merchant_fraud_rate",
    "channel_risk_score",
    "persona_fraud_risk",
    "location_fraud_risk"
]

ENGINEERED_FEATURES = [
    "amount_ratio",
    "amount_z",
    "velocity_intensity"
]

LGBM_BASE_FEATURES = LGBM_BASE_FEATURES + ENGINEERED_FEATURES


In [6]:
LGBM_HYBRID_FEATURES = LGBM_BASE_FEATURES + ["anomaly_score"]


In [7]:
df = pd.read_csv(csv_path)
print("Dataset shape:", df.shape)

Dataset shape: (5000000, 45)


In [8]:
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.dropna(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)

split_ratio = 0.8
split_idx = int(len(df) * split_ratio)

df_train = df.iloc[:split_idx].copy()
df_test  = df.iloc[split_idx:].copy()

print("Train:", df_train.shape, "Test:", df_test.shape)
print("Train fraud rate:", df_train["is_fraud"].mean())
print("Test fraud rate:", df_test["is_fraud"].mean())


Train: (4000000, 45) Test: (1000000, 45)
Train fraud rate: 0.03586475
Test fraud rate: 0.036094


In [9]:
def add_engineered_features(df_part):
    df_part = df_part.copy()

    # 1️⃣ Amount ratio
    df_part["amount_ratio"] = df_part["amount_ngn"] / (df_part["user_avg_txn_amt"] + 1e-6)

    # 2️⃣ Amount z-score
    df_part["amount_z"] = (
        (df_part["amount_ngn"] - df_part["user_avg_txn_amt"])
        / (df_part["user_std_txn_amt"] + 1e-6)
    )

    # 3️⃣ Velocity intensity
    df_part["velocity_intensity"] = (
        df_part["txn_count_last_1h"] / (df_part["user_txn_frequency_24h"] + 1e-6)
    )

    return df_part


df_train = add_engineered_features(df_train)
df_test  = add_engineered_features(df_test)

print("Engineered features added.")


Engineered features added.


In [10]:


required = set(AE_FEATURES + LGBM_BASE_FEATURES + ["is_fraud", "timestamp"])
missing = sorted([c for c in required if c not in df_train.columns])
print("Missing:", missing[:20], "..." if len(missing) > 20 else "")
assert len(missing) == 0, f"Missing columns: {missing}"


Missing: [] 


In [11]:
ae_model = load_model(ae_model_path, compile=False)
scaler = joblib.load(scaler_path)

print("AE loaded.")


AE loaded.


In [12]:
def compute_anomaly_score(df_part: pd.DataFrame, batch_size: int = 4096) -> np.ndarray:
    X = df_part[AE_FEATURES].copy()
    X_scaled = scaler.transform(X)

    recon = ae_model.predict(X_scaled, batch_size=batch_size, verbose=0)
    mse = np.mean(np.square(X_scaled - recon), axis=1)
    return mse

df_train["anomaly_score"] = compute_anomaly_score(df_train)
df_test["anomaly_score"]  = compute_anomaly_score(df_test)

print("anomaly_score added.")
print(df_train["anomaly_score"].describe())


anomaly_score added.
count    4.000000e+06
mean     2.920408e-01
std      3.015197e-01
min      2.632992e-03
25%      1.265267e-01
50%      2.211177e-01
75%      3.765507e-01
max      2.333559e+01
Name: anomaly_score, dtype: float64


In [13]:
y_train = df_train["is_fraud"].astype(int).values
y_test  = df_test["is_fraud"].astype(int).values

X_train_base = df_train[LGBM_BASE_FEATURES].copy()
X_test_base  = df_test[LGBM_BASE_FEATURES].copy()

X_train_hyb = df_train[LGBM_HYBRID_FEATURES].copy()
X_test_hyb  = df_test[LGBM_HYBRID_FEATURES].copy()

print("Base:", X_train_base.shape, X_test_base.shape)
print("Hybrid:", X_train_hyb.shape, X_test_hyb.shape)


Base: (4000000, 24) (1000000, 24)
Hybrid: (4000000, 25) (1000000, 25)


In [14]:
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
scale_pos_weight = neg / max(pos, 1)
print("neg:", neg, "pos:", pos, "scale_pos_weight:", scale_pos_weight)

params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "max_depth": -1,
    "min_data_in_leaf": 200,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "lambda_l2": 1.0,
    #"scale_pos_weight": scale_pos_weight,
    "verbosity": -1,
    "n_jobs": -1,
}


neg: 3856541 pos: 143459 scale_pos_weight: 26.882530897329552


In [15]:
dtrain = lgb.Dataset(X_train_base, label=y_train)
dvalid = lgb.Dataset(X_test_base, label=y_test, reference=dtrain)

baseline_model = lgb.train(
    params,
    dtrain,
    num_boost_round=2000,
    valid_sets=[dtrain, dvalid],
    valid_names=["train", "test"],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(50)]
)


Training until validation scores don't improve for 50 rounds
[50]	train's auc: 0.651133	test's auc: 0.592701
Early stopping, best iteration is:
[9]	train's auc: 0.616461	test's auc: 0.594041


In [16]:
dtrain_h = lgb.Dataset(X_train_hyb, label=y_train)
dvalid_h = lgb.Dataset(X_test_hyb, label=y_test, reference=dtrain_h)

hybrid_model = lgb.train(
    params,
    dtrain_h,
    num_boost_round=2000,
    valid_sets=[dtrain_h, dvalid_h],
    valid_names=["train", "test"],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(50)]
)


Training until validation scores don't improve for 50 rounds
[50]	train's auc: 0.652628	test's auc: 0.593583
Early stopping, best iteration is:
[1]	train's auc: 0.597365	test's auc: 0.594102


In [17]:
def evaluate(model, X_test, y_test, name="model"):
    proba = model.predict(X_test, num_iteration=model.best_iteration)

    auc = roc_auc_score(y_test, proba)
    ap  = average_precision_score(y_test, proba)

    # Threshold seçimi: F1 maksimum
    precision, recall, thresholds = precision_recall_curve(y_test, proba)
    f1s = (2 * precision * recall) / (precision + recall + 1e-12)
    best_idx = np.argmax(f1s)
    best_thr = thresholds[max(best_idx - 1, 0)] if len(thresholds) else 0.5  # güvenli
    y_pred = (proba >= best_thr).astype(int)

    f1 = f1_score(y_test, y_pred)
    p  = precision_score(y_test, y_pred, zero_division=0)
    r  = recall_score(y_test, y_pred, zero_division=0)
    cm = confusion_matrix(y_test, y_pred)

    print(f"\n=== {name} ===")
    print("ROC AUC:", auc)
    print("PR AUC :", ap)
    print("Best thr (F1):", best_thr)
    print("F1:", f1, "Precision:", p, "Recall:", r)
    print("Confusion matrix:\n", cm)

    return {"auc": auc, "ap": ap, "best_thr": float(best_thr), "f1": f1, "precision": p, "recall": r}

baseline_metrics = evaluate(baseline_model, X_test_base, y_test, "Baseline (Behavior only)")
hybrid_metrics   = evaluate(hybrid_model, X_test_hyb, y_test, "Hybrid (Behavior + anomaly_score)")



=== Baseline (Behavior only) ===
ROC AUC: 0.5940411560288424
PR AUC : 0.04415313413185723
Best thr (F1): 0.037067382863988155
F1: 0.08423205434356384 Precision: 0.04397968791376926 Recall: 0.9938770986867623
Confusion matrix:
 [[184107 779799]
 [   221  35873]]

=== Hybrid (Behavior + anomaly_score) ===
ROC AUC: 0.5941016990137806
PR AUC : 0.0440518229368455
Best thr (F1): 0.03606713496188163
F1: 0.0842422846830928 Precision: 0.04398640607555179 Recall: 0.9932952845348257
Confusion matrix:
 [[184688 779218]
 [   242  35852]]


In [23]:
#####################################
RISK_FEATURES = [
    "merchant_fraud_rate",
    "channel_risk_score",
    "persona_fraud_risk",
    "location_fraud_risk"
]

LGBM_BEHAVIOR_RISK = LGBM_BASE_FEATURES + RISK_FEATURES
LGBM_FULL = LGBM_BASE_FEATURES + RISK_FEATURES + ["anomaly_score"]

#X_train_risk = df_train[LGBM_BEHAVIOR_RISK].copy()
#X_test_risk  = df_test[LGBM_BEHAVIOR_RISK].copy()

X_train_risk2 = df_train[RISK_FEATURES].copy()
X_test_risk2  = df_test[RISK_FEATURES].copy()


X_train_full = df_train[LGBM_FULL].copy()
X_test_full  = df_test[LGBM_FULL].copy()


In [24]:
dtrain_risk = lgb.Dataset(X_train_risk2, label=y_train)
dvalid_risk = lgb.Dataset(X_test_risk2, label=y_test)

risk_only_model = lgb.train(
    params,
    dtrain_risk,
    num_boost_round=2000,
    valid_sets=[dtrain_risk, dvalid_risk],
    valid_names=["train", "test"],
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(50)]
)

Training until validation scores don't improve for 50 rounds
[50]	train's auc: 0.548562	test's auc: 0.497546
Early stopping, best iteration is:
[1]	train's auc: 0.506909	test's auc: 0.500615


In [20]:
"""dtrain_r = lgb.Dataset(X_train_risk, label=y_train)
dvalid_r = lgb.Dataset(X_test_risk, label=y_test)

risk_model = lgb.train(
    params,
    dtrain_r,
    num_boost_round=2000,
    valid_sets=[dtrain_r, dvalid_r],
    valid_names=["train", "test"],
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(50)]
)


LightGBMError: Feature (merchant_fraud_rate) appears more than one time.

In [25]:
dtrain_f = lgb.Dataset(X_train_full, label=y_train)
dvalid_f = lgb.Dataset(X_test_full, label=y_test)

full_model = lgb.train(
    params,
    dtrain_f,
    num_boost_round=2000,
    valid_sets=[dtrain_f, dvalid_f],
    valid_names=["train", "test"],
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(50)]
)


LightGBMError: Feature (merchant_fraud_rate) appears more than one time.

In [26]:
##risk_metrics = evaluate(risk_model, X_test_risk, y_test, "Behavior + Risk")
#full_metrics = evaluate(full_model, X_test_full, y_test, "Full (Behavior + Risk + AE)")
risk_only_metrics = evaluate(risk_only_model, X_test_risk, y_test, "Risk only (4 features)")


LightGBMError: The number of features in data (28) is not the same as it was in training data (4).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.

In [24]:
import pandas as pd

imp = pd.DataFrame({
    "feature": X_train_hyb.columns,
    "importance": hybrid_model.feature_importance(importance_type="gain")
}).sort_values("importance", ascending=False)

imp.head(20)


,feature,importance
15,new_device_transaction,7.814353e+06
1,user_avg_txn_amt,1.944609e+04
13,ip_seen_count,1.935728e+04
17,anomaly_score,1.876537e+04
2,user_std_txn_amt,1.759102e+04
0,amount_ngn,1.296648e+04
7,avg_gap_between_txns,1.252072e+04
8,txn_hour,9.814391e+03
6,total_amount_last_1h,8.893408e+03
11,device_seen_count,2.489449e+03


In [ ]:
# LightGBM model
joblib.dump(hybrid_model, "/content/lgbm_model.pkl")

# training metrics (tez için sakla)
report = {
    "baseline": baseline_metrics,
    "hybrid": hybrid_metrics,
    "lgbm_base_features": LGBM_BASE_FEATURES,
    "lgbm_hybrid_features": LGBM_HYBRID_FEATURES,
    "ae_features": AE_FEATURES
}
joblib.dump(report, "/content/training_report.pkl")

print("Saved: lgbm_model.pkl, training_report.pkl")


In [ ]:
from google.colab import files
files.download("/content/lgbm_model.pkl")
files.download("/content/training_report.pkl")
